In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import warnings
warnings.filterwarnings("ignore")

learning_rate = 0.01
momentum = 0.5
log_interval = 10000

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [4]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/tmp', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Resize((14,14)),
                                torchvision.transforms.Lambda(lambda x: torch.flatten(x)),
                             ])), shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/tmp', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Resize((14,14)),
                                torchvision.transforms.Lambda(lambda x: torch.flatten(x)),
                             ])), shuffle=True)

In [1]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)
print(f"example_data.shape: {example_data.shape}")
print(f"example_targets.shape: {example_targets.shape}")

NameError: name 'test_loader' is not defined

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(196, 10)
        self.fc2 = nn.Linear(10, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [7]:
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

In [8]:
train_losses = []
train_counter = []

def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

In [9]:
train(1)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.143008
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.013687
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.142858
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.171771
Train Epoch: 1 [40000/60000 (67%)]	Loss: 4.330003
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.002654


In [106]:
network.eval()

with torch.no_grad():
    pred = network(example_data)
print(f"Prediction: {pred.argmax()}")
print(f"Real Value: {example_targets.item()}")


Prediction: 3
Real Value: 3


In [107]:
example_data.shape

torch.Size([1, 196])

In [108]:
# Generate a dummy input of shape [1, 196]
dummy_input = torch.randn(1, 196)

In [110]:
network.eval()
torch.onnx.export(network, dummy_input, "compl.onnx")

In [96]:
import onnxruntime as ort
import numpy as np

# Load the ONNX model
ort_session = ort.InferenceSession("1l_5.onnx")

# Function to predict using the ONNX model
def predict_with_onnx(model, data_loader):
    for data, targets in data_loader:
        # Convert the PyTorch tensor to a NumPy array
        data_numpy = data.numpy().astype(np.float32)

        # Run the model (input data needs to be in dictionary format)
        ort_inputs = {model.get_inputs()[0].name: data_numpy}
        ort_outs = model.run(None, ort_inputs)

        # Get the prediction and actual label
        predicted = np.argmax(ort_outs[0], axis=1)
        print(f"Prediction: {predicted[0]}")
        print(f"Real Value: {targets.numpy()[0]}")
        break  # For demonstration, break after one batch

# Use the function to predict with the ONNX model
predict_with_onnx(ort_session, test_loader)


Prediction: 4
Real Value: 4


In [88]:
def to_fixed_point(val, bits):
    return round(val * (2**bits))

def mnist_image_to_fixed_point(data):
    return [to_fixed_point(val.item(), 16) for val in data]


def generate_input_cairo(data):
    values = mnist_image_to_fixed_point(data)
    values = [f"FixedTrait::<FP16x16>::new({val}, {'true' if val < 0 else 'false'})" for val in values]
    return ",\n ".join(values)

input_cairo = generate_input_cairo(example_data[0])

with open("mnist_cairo/src/input.cairo", "w") as f:
    f.write("""
use array::{SpanTrait, ArrayTrait};
use orion::operators::tensor::{TensorTrait, FP16x16Tensor, Tensor};
use orion::numbers::{FixedTrait, FP16x16};
fn input() -> Tensor<FP16x16> {
    TensorTrait::<FP16x16>::new(
        array![196].span(),
        array![
    """)
    f.write(input_cairo)
    f.write("""
        ].span()
    )
}
    """)

FileNotFoundError: [Errno 2] No such file or directory: 'mnist_cairo/src/input.cairo'